In [2]:
import glob

import pandas as pd
from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories, get_content_categories_with_numbers, policy_graph
from PIL import Image
from llavaguard.taxonomy.policy_config import local_image_dirs, local_data_dir
from llavaguard.taxonomy.PEGI.PEGI_Graph import get_policy_intro, policy_graph_to_text
import random


im_paths = glob.glob(local_image_dirs['smid'] + '/*.jpg') + glob.glob(local_image_dirs['crawled'] + '/*/*.jpg') + glob.glob(local_image_dirs['synthetic'] + '/*/*.jpg')
random.seed(42)
# Select 50 random elements from im_paths
im_paths = random.sample(im_paths, 2000)
im_paths[0]

categories_with_numbers = get_content_categories_with_numbers(policy_graph)
categories = get_content_categories(policy_graph)


## Only display with explanation

In [ ]:
import glob
from itertools import product
import os
from PIL import Image
import pandas as pd

from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories_with_numbers, get_majority_vote, policy_graph, remove_mutually_exclusive_categories
# check annotations when prompted to evaluate each category separately
models = ["lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "OpenGVLab/InternVL2-Llama3-76B", "OpenGVLab/InternVL2-40B", "OpenGVLab/InternVL2-8B"]
# models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "OpenGVLab/InternVL2-8B"][:]
teq = ['prompt_for_every_content_category', 'prompt_for_every_content_category_describe_image', 'prompt_for_every_content_category_describe_image2']
path_1 = f"{local_data_dir}/data/annotations/auto_generated_annotations"
m_1, t_1 = models[0].split('/')[1], teq[0]


for im_path in im_paths[300:350]:
    sample_path = im_path.split('images/')[-1].split('.jpg')[0]
    sample_id = im_path.split('/')[-1].split('.')[0]
    violations = {}
    for model, t in product(models, [teq[0], teq[2]]):
        name = model.split('/')[1]
        if 'describe_image' in t:
            name += ' with image description'
        violations[name] = []
        missing, invalid = 0, 0
        for category in categories_with_numbers:
            annot_path = os.path.join(path_1, model.split('/')[1], t, sample_path, f"{category}.txt")
            if os.path.exists(annot_path):
                with open(annot_path, 'r') as f:
                    content = f.read()

                    
                    # if 'Yes' in content and 'No' in content:
                    #     print(f"Conflicting entry: {content}")
                    #     invalid += 1
                    #     # violations[annot_name].append(f"Invalid entry: {content}")                    
                    if 'Yes' in content:
                        violations[name].append(category)
                        # if model ==  'OpenGVLab/InternVL2-8B' and t == 'prompt_for_every_content_category_describe_image2':
                        #     print(f"{category}: {content}")
                        # if 'educational' in category.lower():
                        #     print(f"{model} ({category}): {content}")
                    elif 'No' in content or 'Not applicable' in content:
                        pass
                    else:
                        # print(f"{annot_name} invalid: {content}")
                        invalid += 1
                        # violations[annot_name].append(f"Invalid entry: {content}")                
            else:
                # print(f'Missing file {p}')
                missing += 1
        if missing > 0:
            violations[name].append(f"Missing entries: {missing}")
        if invalid > 0:
            violations[name].append(f"Invalid entries: {invalid}")


    im = Image.open(im_path).resize((256, 256))
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)
    mv = get_majority_vote(violations)
    print(f"Majority Vote: {mv}")
    print(f"Cleanded Categories: {remove_mutually_exclusive_categories(mv)}")
    print("====================================="*2)

# display old annotations

In [ ]:
import glob
import os
from PIL import Image

from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories_with_numbers
from llavaguard.taxonomy.PEGI.PEGI_Graph_old import policy_graph as policy_graph_old

# check annotations when prompted to evaluate each category separately
models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "lmms-lab/llama3-llava-next-8b"]
m_1 = models[0].split('/')[1]
teq = ['prompt_for_every_content_category', 'prompt_for_every_content_category_describe_image', 'prompt_for_every_content_category_describe_image2'][2]
path_1 = f"{local_data_dir}/data/annotations/auto_generated_annotations-old"
samples = glob.glob(f"{path_1}/{m_1}/{teq}/*/*", recursive=True)
categories = get_content_categories_with_numbers(policy_graph_old)
print(len(samples))
print(samples[0])
for sample in samples[100:150]:
    sample_path = sample.split(teq)[1]
    sample_id = sample.split('/')[-1]
    violations = {}
    for model in models:
        annot_path = sample.replace(m_1, model.split('/')[1])
        annot_name = model.split('/')[1]
        violations[annot_name] = []
        missing, invalid = 0, 0
        for category in categories:
            category_no_number = category.split('. ')[1].strip()

            p = annot_path + f"/{category}.txt"
            if os.path.exists(p):
                with open(p, 'r') as f:
                    content = f.read()
                    # if 'animal' in category_no_number:
                    #     print(content)
                    if 'Yes' in content:
                        violations[annot_name].append(category)
                    elif 'No' in content or 'Not applicable' in content:
                        pass
                    else:
                        # print(f"{annot_name} invalid: {content}")
                        invalid += 1
                        # violations[annot_name].append(f"Invalid entry: {content}")                
            else:
                # print(f'Missing file {p}')
                missing += 1
        if missing > 0:
            violations[annot_name].append(f"Missing entries: {missing}")
        if invalid > 0:
            violations[annot_name].append(f"Invalid entries: {invalid}")
    im = Image.open(f"{local_data_dir}/data/images/{sample_path}.jpg")
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)
    print(f"Majority Vote: {get_majority_vote(violations)}")
    print("====================================="*2)

# Content Category - One prompt for all content categories
## -> this does not work at all

In [ ]:
# check annotations when prompted to evaluate all content categories in one prompt
from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories


models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "lmms-lab/llama3-llava-next-8b"]
path_2 =  f"{local_data_dir}/data/annotations/auto_generated_annotations/{models[1].split('/')[1]}/prompt_for_all_content_categories"
samples = glob.glob(f"{path_2}/**/*.txt", recursive=True)
print("Number of samples:", len(samples))
print(samples[0])
categories = get_content_categories(policy_graph)

for sample in samples[:]:
    sample_path = sample.split("prompt_for_all_content_categories/")[1].split('.txt')[0]
    sample_id = p.split('/')[-2].split('.txt')[0]
    violations = {}
    for model in models:
        violations[model] = []
        missing = 0
        p2 = sample.replace(path_2.split('/')[-2], model.split('/')[1])
        if os.path.exists(p):
            with open(p2, "r") as f:
                content = f.read()
                # violations[model].append(cats)
                if 'None Applicable' in content or 'No' in content:
                    pass
                else:
                    violations[model].append(content)
                    # for c in categories:
                    #     if c.strip() in content:
                    #         violations[model].append(c.strip())
        else:
            violations[model].append("Missing entries")
    im = Image.open(f"{local_data_dir}/data/images/{sample_path}.jpg")
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)

# Safety Category - One prompt for all safety categories
## -> this also works very bad

In [ ]:
# check annotations when prompted to evaluate all safety categories in one prompt
models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "lmms-lab/llama3-llava-next-8b"]
path_3 =  f"{local_data_dir}/data/annotations/auto_generated_annotations/{models[1].split('/')[1]}/prompt_for_all_safety_categories/"
samples = glob.glob(f"{path_3}/**/*.txt", recursive=True)
categories = get_content_categories(policy_graph)

for sample in samples[:50]:
    sample_path = sample.split("prompt_for_all_safety_categories/")[1].split('.txt')[0]
    sample_id = path_3.split('/')[-2].split('.txt')[0]
    violations = {}
    for model in models:
        violations[model] = []
        missing = 0
        p2 = sample.replace(path_2.split('/')[-2], model.split('/')[1])
        if os.path.exists(p):
            with open(p2, "r") as content:
                content = content.read()
                violations[model] = content

                # if 'None Applicable' in content or 'No' in content:
                #     pass
                # else:
                #     for c in categories:
                #         if c.strip() in content:
                #             violations[model].append(c.strip())
        else:
            violations[model].append("Missing entries")
    im = Image.open(f"{local_data_dir}/data/images/{sample_path}.jpg")
    
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)

In [ ]:
import random


im_paths = glob.glob(local_image_dirs['smid'] + '/*.jpg') + glob.glob(local_image_dirs['crawled'] + '/*/*.jpg') + glob.glob(local_image_dirs['synthetic'] + '/*/*.jpg')
random.seed(42)
im_paths = random.sample(im_paths, 50)
im_path



# Content Category - One prompt for for every safety categories

In [ ]:
import glob
import os
from PIL import Image

from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories_with_numbers, policy_graph

safety_categories = list(policy_graph.keys())
models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "lmms-lab/llama3-llava-next-8b"]
path_1 = f"{local_data_dir}/data/annotations/auto_generated_annotations/{models[-1].split('/')[1]}/prompt_for_every_safety_categories"
samples = glob.glob(f"{path_1}/*/*", recursive=True)
categories = get_content_categories_with_numbers(policy_graph)
print(len(samples))
print(samples[0])
for sample in samples[:10]:
    sample_path = sample.split('prompt_for_every_safety_categories')[1]
    sample_id = sample.split('/')[-1]
    violations = {}
    for model in models:
        model_path = sample.replace(path_1.split('/')[-2], model.split('/')[1])
        violations[model] = []
        missing = 0
        for safety_category, graph_data in policy_graph.items():
            p =  f"{model_path}/{safety_category.replace(' ', '_')}.txt"
            if os.path.exists(p):
                with open(p, 'r') as f:
                    content = f.read()
                    violations[model].append(content)

                    # if 'No' in content:
                    #     pass
                    # else:
                    #     violations[model].append(content)
                
                    # for category in graph_data.keys():
                    #     if category in content:
                    #         violations[model].append(category)
            else:
                missing += 1
        if missing > 0:
            violations[model].append(f"Missing entries: {missing}")
    im = Image.open(f"{local_data_dir}/data/images/{sample_path}.jpg")
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)

# Content category - Individual promts for every category 
## Compare with explanation to no explanation

In [ ]:
import glob
import os
from PIL import Image

from llavaguard.taxonomy.PEGI.PEGI_Graph import get_content_categories_with_numbers, policy_graph

# check annotations when prompted to evaluate each category separately
models = ["Qwen/Qwen2-VL-7B-Instruct", "Qwen/Qwen2-VL-72B-Instruct", "lmms-lab/llava-onevision-qwen2-7b-ov", "lmms-lab/llava-onevision-qwen2-72b-ov", "lmms-lab/llama3-llava-next-8b"]
models = [models[0], models[2]]
m_1 = models[0].split('/')[1]
path_1 = f"{local_data_dir}/data/annotations/tmp"
samples = glob.glob(f"{path_1}/{m_1}/prompt_for_every_content_category/*/*", recursive=True)
categories = get_content_categories_with_numbers(policy_graph)
print(len(samples))
print(samples[0])
for sample in samples[:50]:
    sample_path = sample.split('prompt_for_every_content_category')[1]
    sample_id = sample.split('/')[-1]
    violations = {}
    for model in models:
        for teq in ['prompt_for_every_content_category', 'prompt_for_every_content_category_describe_image2']:
            annot_path = sample.replace(m_1, model.split('/')[1])
            annot_path = annot_path.replace('prompt_for_every_content_category', teq)
            annot_name = f"{model.split('/')[1]} w/ {teq.replace('prompt_for_every_content_category', '')}" if 'describe_image' in annot_path else f"{model.split('/')[1]}"
            violations[annot_name] = []
            missing, invalid = 0, 0
            for category in categories:
                category_no_number = category.split('. ')[1].strip()
                p = annot_path + f"/{category}.txt"
                if os.path.exists(p):
                    with open(p, 'r') as f:
                        content = f.read()                        
                        if 'Yes' in content:
                            violations[annot_name].append(category)
                            if category == '24. Highly Disturbing Content':
                                print(content)
                        elif 'No' in content or 'Not applicable' in content:
                            pass
                        else:
                            # print(f"{annot_name} invalid: {content}")
                            invalid += 1
                            # violations[annot_name].append(f"Invalid entry: {content}")                
                else:
                    # print(f'Missing file {p}')
                    missing += 1
            if missing > 0:
                violations[annot_name].append(f"Missing entries: {missing}")
            if invalid > 0:
                violations[annot_name].append(f"Invalid entries: {invalid}")
    im = Image.open(f"{local_data_dir}/data/images/{sample_path}.jpg")
    im.show()
    print(f"Image: {sample_id}")
    for key, value in violations.items():
        print(f"{key}: {value}")
    print("====================================="*2)